Mostly inspired from atomai_training_data.ipynb

In [1]:
from atomai.utils import create_lattice_mask, extract_patches_
from atomai.transforms import datatransform
import numpy as np
from typing import Optional, Callable, Union, List, Tuple
import argparse
import numpy as np
import torch
import cv2
from scipy import stats, ndimage
from skimage import exposure
from skimage.util import random_noise
from sklearn.model_selection import train_test_split

/opt/anaconda/envs/shared_conda_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lattice_im = np.load('data_pv/LBFO/lattice_img.npy')
xy_atoms_a = np.load('data_pv/LBFO/lattice_coord_a.npy')
xy_atoms_b = np.load('data_pv/LBFO/lattice_coord_b.npy')

In [3]:
lattice_mask_a = create_lattice_mask(lattice_im, xy_atoms_a.T)
lattice_mask_b = create_lattice_mask(lattice_im, xy_atoms_b.T)

In [4]:
print(lattice_im.shape, xy_atoms_a.shape, xy_atoms_b.shape)

(3801, 3801) (2, 64932) (2, 65465)


In [5]:
multiclass = True

In [6]:

if multiclass:
    lattice_mask_backgr = 1 - (lattice_mask_a + lattice_mask_b)
    lattice_mask = np.concatenate(
        (lattice_mask_a[..., None], 
         lattice_mask_b[..., None],
         lattice_mask_backgr[..., None]), # we need to add a background class
        axis=-1)
else:
    lattice_mask = lattice_mask_a + lattice_mask_b
    lattice_mask[lattice_mask>1] = 1

## Generate image-label pairs
images_all, labels_all = extract_patches_(
    lattice_im, lattice_mask, patch_size=256, num_patches=5000, random_state=42)
print(images_all.shape, labels_all.shape)


images_all, images_test_all, labels_all, labels_test_all = train_test_split(
    images_all, labels_all, test_size=0.2, random_state=42)


images_all, images_val_all, labels_all, labels_val_all = train_test_split(
    images_all, labels_all, test_size=0.25, random_state=42)

(5000, 256, 256) (5000, 256, 256, 3)


In [7]:
# For a single class case, we still need to explicitly specify the single channel
labels_all = labels_all[..., None] if np.ndim(labels_all) == 3 else labels_all
# Number of channels in masked data (the training images have a single channel)
ch = labels_all.shape[-1]
# Define image distortion/noise parameters
zoom = 1.1 # zoom factor
poisson = [30, 40] # P noise range (scaled units)
gauss = [20, 100] # G noise range (scaled units)
blur = [1, 40] # Blurring range (scaled units)
contrast = [5, 14] # contrast range (< 10 is brighter, > 10 is darker)
salt_and_pepper = [1, 10] # min/max amount of salted/peppered pixels (scaled units)
# Run the augmentor
imaug = datatransform(
    n_channels=ch, dim_order_in='channel_last', dim_order_out='channel_first', 
    gauss_noise=gauss, poisson_noise=poisson, salt_and_pepper=salt_and_pepper,
    contrast=contrast, blur=blur, zoom=zoom, rotation=True,
    squeeze_channels=True, seed=42)
images_all, labels_all = imaug.run(images_all, labels_all)

print(images_all.shape, labels_all.shape)

(1557, 1, 256, 256) (1557, 256, 256)


In [8]:
imaug = datatransform(
    n_channels=ch, dim_order_in='channel_last', dim_order_out='channel_first', 
    gauss_noise=gauss, poisson_noise=poisson, salt_and_pepper=salt_and_pepper,
    contrast=contrast, blur=blur, zoom=zoom, rotation=True,
    squeeze_channels=True, seed=42)
images_val_all, labels_val_all = imaug.run(images_val_all, labels_val_all)

In [9]:
print(images_val_all.shape, labels_val_all.shape)

(485, 1, 256, 256) (485, 256, 256)


In [10]:
#preserve everything about test, just convert to format required
imaug = datatransform(n_channels=ch, dim_order_in='channel_last', dim_order_out='channel_first',
    squeeze_channels=True, seed=42)
images_test_all, labels_test_all = imaug.run(images_test_all, labels_test_all)

In [11]:
print(images_test_all.shape, labels_test_all.shape)

(787, 1, 256, 256) (787, 256, 256)


In [12]:
path='data_out_pv/'
np.save(path+'X_train.npy',images_all)
np.save(path+'y_train.npy',labels_all)
np.save(path+'X_val.npy',images_val_all)
np.save(path+'y_val.npy',labels_val_all)
np.save(path+'X_test.npy',images_test_all)
np.save(path+'y_test.npy',labels_test_all)